In [226]:
import pandas as pd
import numpy as np
import random
import re
import os,sys
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.utils import class_weight
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [227]:

print(device)

cpu


In [21]:
train_path = "data/train"
dev_path = "data/dev"
test_path = "data/test"
unk = "<unk>"
pad = "<pad>"
num = "<num>"
max_len = 128
numbers = ['one','two','three','four','five', 'six','seven','eight','nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'zero', 'hundred', 'thousand', 'million', 'billion', 'trillion', 'quadrillion', 'quintillion', 'sextillion', 'septillion', 'octillion', 'nonillion', 'decillion']

In [22]:
def read_data(filename):
    f=open(os.path.join(sys.path[0],filename),"r")
    lines=f.read().splitlines()
    lst_sent =[]
    for x in lines:
        y = x.split()
        #print(x.split())
        if len(y)==3:
            lst_sent.append(y)
    f.close()
#print(result)

    return lst_sent
    

In [23]:
train_set = read_data(train_path)
dev_set = read_data(dev_path)

In [24]:
def is_number(s):
        if "," in s:
            s = s.replace(",", "")
        if ":" in s:
            s = s.replace(":", "")
        if "\/" in s:
            s = s.replace("\/", "")
        if "-" in s:
            s = s.replace("-", "")
        if "/" in s:
            s = s.replace("/", "")
        if s.lower() in numbers:
            #print(s)
            return True
        elif s.isdecimal():
            return True
        else:
            return False
    

In [25]:
v = is_number('12,:/2')
print(v)

True


In [183]:
vocab = {}
tag_to_idx = {}
idx_to_tag = {}
just_tags = []
idx = 0
threshold = 3

In [184]:
for line in train_set:
    word = line[1]
    tag = line[2]
    if is_number(word):
        word = num
    if word in vocab:
        vocab[word] += 1
    else:
        vocab[word] = 1
    if tag not in tag_to_idx:
        tag_to_idx[tag] = idx
        idx_to_tag[idx] = tag
        idx += 1
    just_tags.append(tag_to_idx[tag])
for word, freq in  list(vocab.items()):
    if freq < threshold:
        del vocab[word]
vocab["<unk>"] = 1

In [190]:
print(len(vocab))

7703


In [191]:
word_to_idx = {}
idx_to_word = {}
idx = 0
for word in vocab:
    if word not in word_to_idx:
        word_to_idx[word] = idx
        idx_to_word[idx] = word
        idx += 1
word_to_idx[pad] = idx
idx_to_word[idx] = pad
pad_idx = word_to_idx[pad]

In [192]:
print(word_to_idx['EU'])
print(len(vocab))
print(pad_idx)

0
7703
7703


In [193]:
def dataset_creation(sentence):
    global word_to_idx, tag_to_idx, unk, max_len
    #sen_len = len(sentence)
    pad_len = max_len - len(sentence)
    words = np.array([])
    tags = np.array([])
    #print('sen:',sentence)
    for idx, word, tag in sentence:
        if word not in word_to_idx:
            if is_number(word):
                word = num

            else:
                word = unk
        #print('word:',word)
        words= np.append(words,word_to_idx[word])
        tags= np.append(tags,tag_to_idx[tag])
    #words = np.array(words)
    #tags = np.array(tags)
    #print('w,t:',words,tags)
    pad_seq = pad_idx * np.ones(pad_len)
    pad_tag = -1 * np.ones(pad_len)
    words = np.concatenate((words, pad_seq), axis = 0)
    tags = np.concatenate((tags, pad_tag), axis = 0)
    return words, tags

In [194]:
def create_lstm_ip(dataset):
    x_lstm = []
    y_lstm = []
    sentence = []
    sentence.append(dataset[0])
    j = 0
    for i in range(1,len(dataset)):
        #print(sentence)
        
        
        if dataset[i][0] != '1':
            sentence.append(dataset[i])
        else:
            x, y = dataset_creation(sentence)
            #print('xy:',x,y)
            x_lstm.append(x)
            y_lstm.append(y)
            #print(x_lstm)
            sentence = []
            sentence.append(dataset[i])

    #print(x_lstm)  
    return np.array(x_lstm), np.array(y_lstm)

In [195]:
x_lstm_train, y_lstm_train = create_lstm_ip(train_set)
x_lstm_dev, y_lstm_dev = create_lstm_ip(dev_set)

In [196]:
print(type(x_lstm_train))

<class 'numpy.ndarray'>


In [197]:
print(x_lstm_train[6])

[7.100e+01 1.500e+01 7.200e+01 4.700e+01 7.300e+01 4.800e+01 7.400e+01
 7.500e+01 7.600e+01 1.800e+01 4.800e+01 7.700e+01 7.800e+01 7.900e+01
 4.800e+01 8.000e+01 1.800e+01 4.000e+01 3.000e+01 8.100e+01 8.200e+01
 3.500e+01 1.300e+01 3.600e+01 7.000e+00 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03 7.703e+03
 7.703

In [198]:

class data(Dataset):
    def __init__(self, inputs, transform = None):
        self.data = inputs
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        inputs = self.data[index][0]
        label = self.data[index][1]
        if self.transform is not None:
            inputs = self.transform(inputs)
            
        return inputs, label

In [199]:
x_lstm_train, x_lstm_dev = torch.LongTensor(x_lstm_train), torch.LongTensor(x_lstm_dev)
y_lstm_train, y_lstm_dev = torch.LongTensor(y_lstm_train), torch.LongTensor(y_lstm_dev)

In [200]:
print(tag_to_idx)

{'B-ORG': 0, 'O': 1, 'B-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-ORG': 6, 'I-MISC': 7, 'I-LOC': 8}


In [201]:
print(len(x_lstm_train))

14986


In [202]:
lstm_train_dataset = TensorDataset(x_lstm_train, y_lstm_train)
#print(len(lstm_train_dataset))
lstm_train_dataset = data(lstm_train_dataset)

lstm_dev_dataset = TensorDataset(x_lstm_dev, y_lstm_dev)
lstm_dev_dataset = data(lstm_dev_dataset)

lstm_train_loader = DataLoader(lstm_train_dataset, batch_size = 8, drop_last = True, shuffle = True)
print(lstm_train_loader)
lstm_dev_loader = DataLoader(lstm_dev_dataset, batch_size = 8, drop_last = True, shuffle = True)

In [205]:
input_dim = len(word_to_idx)
embed_dim = 100
hidden_dim = 256
linear_dim = 128
output_dim = len(tag_to_idx)
pad_idx = word_to_idx[pad]
class_weights = class_weight.compute_class_weight('balanced', np.unique(just_tags), just_tags)

In [206]:
print(tag_to_idx)
print(class_weights)

{'B-ORG': 0, 'O': 1, 'B-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-ORG': 6, 'I-MISC': 7, 'I-LOC': 8}
[ 3.59589727  0.13329307  6.61130502  3.44388889  5.01980271  3.1834267
  6.13651908 19.67936508 19.64534716]


In [218]:
class bLSTM(torch.nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, linear_dim, output_dim, pad_idx):
        super(bLSTM, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings = input_dim, embedding_dim = embed_dim, padding_idx = pad_idx)
        self.LSTM_dropout = torch.nn.Dropout(0.33)
        self.lstm = torch.nn.LSTM(input_size = embed_dim, hidden_size = hidden_dim, num_layers = 1, bidirectional = True)
        self.fc = torch.nn.Linear(hidden_dim * 2, linear_dim)
        self.out = torch.nn.Linear(linear_dim, output_dim)
        self.elu = torch.nn.ELU()
    def forward(self, x):
        embedding_out = self.LSTM_dropout(self.embedding(x))
        lstm_out, _ = self.lstm(embedding_out)
        fc_in = self.LSTM_dropout(lstm_out)
        fc_out = self.fc(fc_in)
        output = self.out(self.elu(fc_out))
        return torch.softmax(output)
    def init_weights(self):
        for name, param in self.named_parameters():
            torch.nn.init.normal_(param.data, mean=0, std=0.1)

    def init_embeddings(self, word_pad_idx):
        self.embedding.weight.data[pad_idx] = torch.zeros(self.embedding_dim)

In [217]:
del model, optimizer, loss_fn

In [225]:
model = bLSTM(input_dim, embed_dim, hidden_dim, linear_dim, output_dim, pad_idx)
print(model)

bLSTM(
  (embedding): Embedding(7704, 100, padding_idx=7703)
  (LSTM_dropout): Dropout(p=0.33, inplace=False)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=9, bias=True)
  (elu): ELU(alpha=1.0)
)


In [221]:
class_weights = torch.FloatTensor(class_weights)
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights, ignore_index = -1)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [223]:
epochs = 1
dev_min_loss = np.inf
train_loader = lstm_train_loader
dev_loader = lstm_dev_loader

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    dev_loss = 0.0
    batch = 0
    j = 0
    for inputs, target in train_loader:
        if j==2:
            break
        j+=1
        print(batch, end = "\r")
        batch += 1
        inputs, target = inputs, target
        print('ip:',inputs,' t:',target)
        optimizer.zero_grad()
        output = model(inputs)
        print('op:',output)
        output = output.view(-1, output.shape[-1])
        target = target.view(-1)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    for inputs, target in dev_loader:
        if j==2:
            break
        j+=1
        inputs, target = inputs, target
        #inputs, target = inputs, target
        output = model(inputs)
        output = output.view(-1, output.shape[-1])
        target = target.view(-1)
        loss = loss_fn(output, target)
        dev_loss += loss.item()
    
    train_loss = train_loss/len(train_loader.dataset)
    dev_loss = dev_loss/len(dev_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tDev Set Loss: {:.6f}'.format(epoch+1, train_loss,dev_loss))
    if dev_loss <= dev_min_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(dev_min_loss, dev_loss))
        torch.save(model.state_dict(), 'simple_blstm.pt')
        dev_min_loss = dev_loss

ip: tensor([[7702,  544,  261,  ..., 7703, 7703, 7703],
        [  71,   15,  346,  ..., 7703, 7703, 7703],
        [5294,   48, 2785,  ..., 7703, 7703, 7703],
        ...,
        [7654, 7655, 1066,  ..., 7703, 7703, 7703],
        [7702,  103, 1635,  ..., 7703, 7703, 7703],
        [ 988,   11, 7703,  ..., 7703, 7703, 7703]])  t: tensor([[ 3,  1,  1,  ..., -1, -1, -1],
        [ 1,  1,  1,  ..., -1, -1, -1],
        [ 3,  1,  1,  ..., -1, -1, -1],
        ...,
        [ 1,  1,  1,  ..., -1, -1, -1],
        [ 1,  1,  3,  ..., -1, -1, -1],
        [ 5,  1, -1,  ..., -1, -1, -1]])
op: tensor([[[-6.6658e-02,  6.8406e-02, -7.1835e-02,  ..., -4.5560e-02,
          -5.1182e-02, -3.1826e-02],
         [-7.4208e-02, -1.3199e-02, -5.8881e-02,  ..., -8.0428e-02,
          -7.9540e-02, -6.8284e-02],
         [-9.9876e-03,  2.0948e-02,  5.3568e-02,  ..., -1.1345e-01,
          -2.9239e-02, -2.0851e-02],
         ...,
         [-6.6707e-02,  3.7815e-02, -3.6454e-02,  ..., -6.4429e-02,
          -

In [49]:
def print_sentence(x):
    global idx_to_word
    sent = ''
    for idx in x:
        if idx != pad_idx:
            sent = sent + " " + idx_to_word[idx]
    print(sent)

In [50]:
def create_op_file(x, y, model, dataset):
    model.load_state_dict(torch.load('simple_blstm.pt'))
    model.eval()
    line_num = 0
    global idx_to_word, idx_to_tag
    with torch.no_grad():
        with open("output", "w") as fp:
            for i in range(len(x)):
                print(i, end = "\r")
                idx = 1
                ip = x[i]
                ip = torch.unsqueeze(ip, 0)
                op = model(ip)
                op = op.view(-1, op.shape[-1])
                target = y[i]
                _, pred = torch.max(op, 1)
                for j in range(len(target)):
                    if target[j] == -1:
                        fp.write("\n")
                        break
                    pred_tag = int(pred[j].item())
                    targ_tag = int(target[j].item())
                    z = int(x[i][j].item())
                    z = idx_to_word[z]
                    if z != dataset[line_num][1]:
                        z = dataset[line_num][1]
                    fp.write("{} {} {} {}\n".format(idx, z, idx_to_tag[targ_tag], idx_to_tag[pred_tag]))
                    idx += 1
                    line_num += 1

In [51]:
model = bLSTM(input_dim, embed_dim, hidden_dim, linear_dim, output_dim, pad_idx)
create_op_file(x_lstm_train, y_lstm_train, model, train_set)